<a href="https://colab.research.google.com/github/yekcel/AI-Agent-MRI-Image-Defacing/blob/main/Multi_agnet_deface_MRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q ultralytics opencv-python google-generativeai pillow numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.5 MB/s eta 0:00:00


In [2]:
import os
from pathlib import Path
import requests

# Create directories
Path('models').mkdir(exist_ok=True)
Path('input_images').mkdir(exist_ok=True)
Path('output_images').mkdir(exist_ok=True)

# Download your trained YOLO model from GitHub
YOLO_MODEL_URL = "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/blob/e5443a0bd32dcb86bf21142311e8d8231e1bef61/model_outputs/best_model.pt"
MODEL_PATH = "models/yolov8n-face.pt"

if not os.path.exists(MODEL_PATH):
    print("Downloading YOLO model...")
    response = requests.get(YOLO_MODEL_URL)
    with open(MODEL_PATH, 'wb') as f:
        f.write(response.content)
    print("✅ Model downloaded!")
else:
    print("✅ Model already exists!")


✅ Model downloaded!


In [4]:
import google.generativeai as genai
from google.colab import userdata

# Get API key from Colab secrets
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

print("✅ Gemini API configured!")


✅ Gemini API configured!


In [5]:
import cv2
import numpy as np

class IrreversibleDefacer:

    @staticmethod
    def expand_bbox(bbox, image_shape, expansion=1.2):
        x1, y1, x2, y2 = bbox
        h, w = image_shape[:2]

        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        width = x2 - x1
        height = y2 - y1

        new_width = width * expansion
        new_height = height * expansion

        new_x1 = max(0, int(center_x - new_width / 2))
        new_y1 = max(0, int(center_y - new_height / 2))
        new_x2 = min(w, int(center_x + new_width / 2))
        new_y2 = min(h, int(center_y + new_height / 2))

        return (new_x1, new_y1, new_x2, new_y2)

    @staticmethod
    def inpaint_deface(image, bbox):
        """Gentle: Inpainting with surrounding texture"""
        img = image.copy()
        x1, y1, x2, y2 = IrreversibleDefacer.expand_bbox(bbox, img.shape, 1.2)

        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        mask[y1:y2, x1:x2] = 255

        result = cv2.inpaint(img, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
        return result

    @staticmethod
    def random_texture_deface(image, bbox):
        """Moderate: Random texture from background"""
        img = image.copy()
        x1, y1, x2, y2 = IrreversibleDefacer.expand_bbox(bbox, img.shape, 1.3)

        h, w = img.shape[:2]
        face_h, face_w = y2 - y1, x2 - x1

        bg_regions = []
        if y1 > face_h:
            bg_regions.append(img[y1-face_h:y1, :])
        if x1 > face_w:
            bg_regions.append(img[:, x1-face_w:x1])

        if bg_regions:
            bg_pixels = np.concatenate([r.flatten() for r in bg_regions])
            random_texture = np.random.choice(bg_pixels, size=(face_h, face_w))
        else:
            random_texture = np.random.randint(0, 256, size=(face_h, face_w), dtype=np.uint8)

        img[y1:y2, x1:x2] = random_texture
        return img

    @staticmethod
    def blackbox_deface(image, bbox):
        """Maximum: Complete black box"""
        img = image.copy()
        x1, y1, x2, y2 = IrreversibleDefacer.expand_bbox(bbox, img.shape, 1.4)
        img[y1:y2, x1:x2] = 0
        return img

    @staticmethod
    def skull_strip_deface(image, bbox):
        """Nuclear: Remove entire head region"""
        img = image.copy()
        x1, y1, x2, y2 = IrreversibleDefacer.expand_bbox(bbox, img.shape, 2.0)
        img[y1:y2, x1:x2] = 0
        return img

print("✅ Defacing strategies defined!")


✅ Defacing strategies defined!


In [6]:
from ultralytics import YOLO

class DetectorAgent:
    def __init__(self, model_path='models/yolov8n-face.pt'):
        self.yolo = YOLO(model_path)
        self.name = "DetectorAgent"

    def run(self, image):
        """Detect faces in the image"""
        detections = []
        results = self.yolo(image, conf=0.5, verbose=False)

        if len(results[0].boxes) > 0:
            boxes = results[0].boxes
            for i in range(len(boxes)):
                x1, y1, x2, y2 = boxes.xyxy[i].cpu().numpy()
                confidence = float(boxes.conf[i])

                detections.append({
                    'bbox': (int(x1), int(y1), int(x2), int(y2)),
                    'confidence': confidence
                })

        return {
            'detections': detections,
            'num_faces': len(detections),
            'status': 'success'
        }

print("✅ DetectorAgent ready!")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ DetectorAgent ready!


In [7]:
class DefacerAgent:
    def __init__(self):
        self.name = "DefacerAgent"
        self.strategies = {
            'gentle': IrreversibleDefacer.inpaint_deface,
            'moderate': IrreversibleDefacer.random_texture_deface,
            'maximum': IrreversibleDefacer.blackbox_deface,
            'skull_strip': IrreversibleDefacer.skull_strip_deface
        }

    def run(self, image, detections, strategy='gentle'):
        """Apply defacing strategy to detected faces"""
        defaced_image = image.copy()

        for detection in detections:
            bbox = detection['bbox']
            deface_func = self.strategies[strategy]
            defaced_image = deface_func(defaced_image, bbox)

        return {
            'defaced_image': defaced_image,
            'strategy_used': strategy,
            'status': 'success'
        }

print("✅ DefacerAgent ready!")


✅ DefacerAgent ready!


In [8]:
class ValidatorAgent:
    def __init__(self, model_path='models/yolov8n-face.pt', threshold=0.3):
        self.yolo = YOLO(model_path)
        self.threshold = threshold
        self.name = "ValidatorAgent"

    def run(self, image):
        """Validate defacing quality - good enough if confidence < threshold"""
        results = self.yolo(image, conf=0.1, verbose=False)

        if len(results[0].boxes) == 0:
            return {
                'is_valid': True,
                'max_confidence': 0.0,
                'num_faces': 0,
                'message': 'No face detected - Perfect!',
                'status': 'success'
            }

        max_confidence = float(results[0].boxes.conf.max())
        is_valid = max_confidence < self.threshold

        return {
            'is_valid': is_valid,
            'max_confidence': max_confidence,
            'num_faces': len(results[0].boxes),
            'message': f"Max confidence: {max_confidence:.3f} ({'Valid' if is_valid else 'Need retry'})",
            'status': 'success'
        }

print("✅ ValidatorAgent ready!")


✅ ValidatorAgent ready!


In [9]:
class GeminiStrategyAgent:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-2.0-flash-exp')
        self.name = "GeminiStrategyAgent"

    def run(self, validation_result, iteration):
        """Decide next defacing strategy using Gemini"""
        confidence = validation_result.get('max_confidence', 0)
        num_faces = validation_result.get('num_faces', 0)

        prompt = f"""You are a defacing strategy optimizer for medical MRI images.

Current Status:
- Iteration: {iteration}/4
- Remaining faces detected: {num_faces}
- Max detection confidence: {confidence:.3f}

Available strategies (from gentle to aggressive):
1. gentle: Inpainting (natural texture fill)
2. moderate: Random texture from background
3. maximum: Complete black box
4. skull_strip: Nuclear option - removes entire head region

Decision Rules:
- If confidence > 0.7 → Use "maximum"
- If confidence 0.4-0.7 → Use "moderate"
- If confidence < 0.4 → Use "gentle"
- If iteration >= 3 → Use "skull_strip"

Respond with ONLY ONE WORD - the strategy name (gentle/moderate/maximum/skull_strip).
"""

        try:
            response = self.model.generate_content(prompt)
            strategy = response.text.strip().lower()

            valid_strategies = ['gentle', 'moderate', 'maximum', 'skull_strip']
            if strategy not in valid_strategies:
                strategy = 'moderate'

            return {
                'strategy': strategy,
                'reasoning': f"Confidence {confidence:.3f} at iteration {iteration}",
                'status': 'success'
            }
        except Exception as e:
            return {
                'strategy': 'moderate',
                'reasoning': f"Error: {str(e)} - using fallback",
                'status': 'fallback'
            }

print("✅ GeminiStrategyAgent ready!")


✅ GeminiStrategyAgent ready!


In [10]:
class MultiAgentOrchestrator:
    def __init__(self, max_iterations=4, validation_threshold=0.3):
        self.detector = DetectorAgent()
        self.defacer = DefacerAgent()
        self.validator = ValidatorAgent(threshold=validation_threshold)
        self.strategy_agent = GeminiStrategyAgent()
        self.max_iterations = max_iterations

        print(f"🎯 Orchestrator initialized with {max_iterations} max iterations")

    def run(self, image_path):
        """Main orchestration loop"""
        print(f"\n{'='*60}")
        print(f"🚀 Processing: {Path(image_path).name}")
        print(f"{'='*60}\n")

        # Step 1: Load image
        image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            return {'status': 'error', 'message': 'Failed to load image'}

        # Step 2: Detect faces
        print(f"[{self.detector.name}] Detecting faces...")
        detection_result = self.detector.run(image)

        if detection_result['num_faces'] == 0:
            print("✅ No faces detected - image is clean!")
            return {
                'status': 'no_faces_detected',
                'message': 'No faces found in image',
                'defaced_image': image,
                'iterations': 0,
                'strategies_used': []
            }

        print(f"   Found {detection_result['num_faces']} face(s)")

        # Step 3: Iterative defacing loop
        detections = detection_result['detections']
        current_image = image.copy()
        strategy_history = []

        for iteration in range(1, self.max_iterations + 1):
            print(f"\n--- Iteration {iteration}/{self.max_iterations} ---")

            # Decide strategy
            if iteration == 1:
                strategy = 'gentle'
                print(f"[{self.strategy_agent.name}] Starting with '{strategy}' strategy")
            else:
                print(f"[{self.strategy_agent.name}] Analyzing validation result...")
                strategy_result = self.strategy_agent.run(validation_result, iteration)
                strategy = strategy_result['strategy']
                print(f"   Decision: '{strategy}' ({strategy_result['reasoning']})")

            strategy_history.append(strategy)

            # Apply defacing
            print(f"[{self.defacer.name}] Applying '{strategy}' defacing...")
            deface_result = self.defacer.run(current_image, detections, strategy)
            current_image = deface_result['defaced_image']

            # Validate result
            print(f"[{self.validator.name}] Validating defaced image...")
            validation_result = self.validator.run(current_image)
            print(f"   {validation_result['message']}")

            if validation_result['is_valid']:
                print(f"\n✅ SUCCESS! Defacing completed in {iteration} iteration(s)")
                break
        else:
            print(f"\n⚠️  Reached max iterations ({self.max_iterations})")

        return {
            'status': 'success' if validation_result['is_valid'] else 'max_iterations_reached',
            'iterations': iteration,
            'strategies_used': strategy_history,
            'final_confidence': validation_result['max_confidence'],
            'final_faces_detected': validation_result['num_faces'],
            'defaced_image': current_image
        }

print("✅ MultiAgentOrchestrator ready!")


✅ MultiAgentOrchestrator ready!
